# HDP Model

In [18]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # Author: don't skip this
# pyLDAvis.gensim.prepare

# I think i need another one:
import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis

# Plots
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [19]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')
stop_words.extend(['tenlastelegging\\n\\n\\naan', '\\n\\n\\n\\n\\n\\n', '\\n\\n\\n\\n\\n', '\\n\\n', 'een', '\\n'])

# Data

In [20]:
df = pd.read_csv('dataset_10012022.csv', index_col=1) 

df_be = df[df['bewijs'] != '[]'] #bewijs
df_te = df[df['tll'] != '[]'] #tenlastelegging
df_be

,Unnamed: 0,ecli,subject,spatial,case_nr,date,bewijs,tll
filename,,,,,,,,
ECLI_NL_GHSHE_2021_1801.xml,9,ECLI:NL:GHSHE:2021:1801,Strafrecht,'s-Hertogenbosch,20-003457-18,2021-02-10,['[\'\\n\\n\\n\\n\\n\\nbewijsmiddelen\\n\\n\\n...,[]
ECLI_NL_GHARL_2020_820.xml,10,ECLI:NL:GHARL:2020:820,Strafrecht,Leeuwarden,21-001899-19,2020-01-30,"[""['\\n\\n\\n\\n\\n\\nbewijsmiddelen, feiten e...","[""[' tenlastelegging\\n\\n\\naan verdachte is ..."
ECLI_NL_RBROT_2021_2966.xml,24,ECLI:NL:RBROT:2021:2966,Strafrecht,Rotterdam,10.295806.20,2021-02-09,['[\'\\n\\nbewijswaardering\\n\\n\\n4.1.1.\\n\...,[]
ECLI_NL_RBMNE_2019_6577.xml,28,ECLI:NL:RBMNE:2019:6577,Strafrecht,Utrecht,16/018445-18,2019-01-18,"[""['\\n\\n\\nbewijsmiddelen\\n\\n\\n\\n ...","[""['\\n\\n2tenlastelegging\\n\\n\\nde tenlaste..."
ECLI_NL_GHAMS_2018_185.xml,32,ECLI:NL:GHAMS:2018:185,Strafrecht,Amsterdam,23-002003-17,2018-01-24,"[""['\\n\\n\\n\\n\\n\\nbewijsverweren\\n\\n\\n\...",[]
...,...,...,...,...,...,...,...,...
ECLI_NL_RBLIM_2019_553.xml,3415,ECLI:NL:RBLIM:2019:553,Strafrecht,Maastricht,03/700387-18,2019-01-23,"[""['\\n\\n\\n\\n\\nbewijsoverweging\\n\\n\\n\\...","[""[' tenlastelegging\\n\\nde tenlastelegging i..."
ECLI_NL_GHSHE_2020_1165.xml,3418,ECLI:NL:GHSHE:2020:1165,Strafrecht,'s-Hertogenbosch,20-002048-18,2020-01-29,"[""['\\n\\n\\n\\n\\n\\nbewijsmiddelen\\n\\n\\n\...","['[""\\n\\n\\n\\n\\n\\ntenlastelegging\\n\\n\\n..."
ECLI_NL_RBMNE_2019_275.xml,3422,ECLI:NL:RBMNE:2019:275,Strafrecht; Materieel strafrecht,Lelystad,16/659548-18,2019-01-25,"[""['\\n\\n\\n\\n\\nbewijsmiddelen\\n\\n\\n\\n ...","[""['\\n\\n2tenlastelegging\\n\\n\\nde tenlaste..."


# Tokenize 

In [21]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df_be['bewijs']))

print(data_words[:1])

[['nbewijsmiddelen', 'nindien', 'tegen', 'dit', 'verkorte', 'arrest', 'beroep', 'in', 'cassatie', 'wordt', 'ingesteld', 'worden', 'de', 'door', 'het', 'hof', 'gebruikte', 'bewijsmiddelen', 'die', 'redengevend', 'zijn', 'voor', 'de', 'opgenomen', 'in', 'een', 'aanvulling', 'op', 'het', 'arrest', 'deze', 'aanvulling', 'wordt', 'dan', 'aan', 'dit', 'arrest', 'gehecht', 'nde', 'beslissing', 'dat', 'het', 'door', 'de', 'verdachte', 'is', 'begaan', 'berust', 'op', 'de', 'feiten', 'en', 'omstandigheden', 'als', 'vervat', 'in', 'de', 'hierboven', 'bedoelde', 'bewijsmiddelen', 'in', 'onderlinge', 'samenhang', 'beschouwd', 'nstrafbaarheid', 'van', 'het', 'nhet', 'wordt', 'als', 'volgt', 'gekwalificeerd', 'novertreding', 'van', 'een', 'voorschrift', 'gesteld', 'krachtens', 'artikel', 'eerste', 'lid', 'van', 'de', 'wet', 'milieubeheer', 'ner', 'zijn', 'geen', 'feiten', 'of', 'omstandigheden', 'aannemelijk', 'geworden', 'die', 'de', 'strafbaarheid', 'van', 'het', 'uitsluiten', 'het', 'feit', 'is', 

# Creating Bigram and Trigram Models

In [22]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['nbewijsmiddelen_nindien', 'tegen', 'dit', 'verkorte_arrest', 'beroep', 'in', 'cassatie', 'wordt', 'ingesteld', 'worden', 'de', 'door', 'het', 'hof', 'gebruikte', 'bewijsmiddelen', 'die', 'redengevend', 'zijn', 'voor', 'de', 'opgenomen', 'in', 'een', 'aanvulling', 'op', 'het', 'arrest', 'deze', 'aanvulling', 'wordt', 'dan', 'aan', 'dit', 'arrest_gehecht', 'nde', 'beslissing', 'dat', 'het', 'door', 'de', 'verdachte', 'is', 'begaan_berust', 'op', 'de', 'feiten', 'en', 'omstandigheden', 'als', 'vervat', 'in', 'de', 'hierboven_bedoelde', 'bewijsmiddelen', 'in', 'onderlinge_samenhang_beschouwd', 'nstrafbaarheid', 'van', 'het', 'nhet', 'wordt', 'als', 'volgt_gekwalificeerd', 'novertreding', 'van', 'een', 'voorschrift', 'gesteld', 'krachtens', 'artikel', 'eerste_lid', 'van', 'de', 'wet_milieubeheer', 'ner', 'zijn', 'geen', 'feiten', 'of', 'omstandigheden_aannemelijk_geworden', 'die', 'de', 'strafbaarheid', 'van', 'het', 'uitsluiten', 'het', 'feit', 'is', 'strafbaar_nstrafbaarheid', 'van', 'd

# Remove Stopwords, Make Bigrams and Lemmatize

In [23]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('nl', disable=['parser', 'ner'])

nlp = spacy.load("nl_core_news_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['verkorte_arrest', 'beroep', 'cassatie', 'instellen', 'hof', 'gebruiken', 'bewijsmiddel', 'redengevend', 'opnemen', 'aanvulling', 'arrest', 'aanvulling', 'nd', 'beslissing', 'verdenken', 'begaan_berust', 'feit', 'omstandigheid', 'vervatten', 'hierboven_bedoelde', 'bewijsmiddel', 'onderlinge_samenhang', 'beschouwen', 'nstrafbaarheid', 'novertreding', 'voorschrift', 'stellen', 'artikel', 'één', 'lid', 'feit', 'omstandigheid', 'aannemelijk_geworden', 'strafbaarheid', 'uitsluiten', 'feit', 'verdenken', 'feit', 'omstandigheid', 'aannemelijk_geworden', 'strafbaarheid', 'verdenken', 'verdenken', 'daarom', 'strafbaar', 'hiervoor', 'leggen_sanctie', 'hof', 'letten', 'aard', 'ernst', 'bewijzen', 'verklaren', 'omstandigheid', 'waaronder', 'persoon', 'verdenken', 'ander', 'onderzoek', 'ter_terechtzitting', 'voren', 'komen', 'daarnaast', 'letten', 'verhouding', 'strafbaar', 'feit', 'uitdrukking_komt', 'hierop_gestelen', 'wettelijk_strafmaximum', 'straf', 'soortgelijk', 'feit', 'opleggen', 'nd', '

#  Create the Dictionary and Corpus needed for Topic Modeling

In [25]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 7), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 2), (23, 1), (24, 2), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 8), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 14), (67, 3), (68, 2), (69, 1), (70, 2), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 3), (81, 1), (82, 1), (83, 1), (84, 3), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 5), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1

In [26]:
# Id word
id2word[0]

'aannemelijk_geworden'

# Building the Topic Model

In [27]:
# Build HDP model
from gensim.models import HdpModel
hdp_model = HdpModel(corpus=corpus,
                     id2word=id2word)

# View the topics in HDP model

In [28]:
# Print the Keyword in the 10 topics
pprint(hdp_model.print_topics())
doc_hdp = hdp_model[corpus]

[(0,
  '0.042*verdenken + 0.019*slachtoffer + 0.009*rechtbank + 0.008*zien + '
  '0.008*medeverdacht + 0.008*feit + 0.008*verklaren + 0.008*hof + 0.008*naam '
  '+ 0.007*bijnaam'),
 (1,
  '0.041*verdenken + 0.017*slachtoffer + 0.012*rechtbank + 0.011*naam + '
  '0.008*verklaren + 0.008*medeverdacht + 0.008*feit + 0.007*zien + '
  '0.006*verklaring + 0.005*komen'),
 (2,
  '0.033*verdenken + 0.018*naam + 0.014*medeverdacht + 0.010*hof + '
  '0.010*verklaren + 0.007*geld + 0.007*benadelen + 0.007*rechtbank + '
  '0.006*blijken + 0.006*feit'),
 (3,
  '0.028*verdenken + 0.020*naam + 0.013*benadelen + 0.009*slachtoffer + '
  '0.008*zegel + 0.008*rechtbank + 0.008*medeverdacht + 0.006*verklaren + '
  '0.005*feit + 0.005*komen'),
 (4,
  '0.038*verdenken + 0.026*medeverdacht + 0.025*zeggen + 0.017*gaan + '
  '0.010*uur + 0.009*bellen + 0.008*weten + 0.007*goed + 0.006*komen + '
  '0.006*medeverdachen'),
 (5,
  '0.043*verdenken + 0.012*rechtbank + 0.009*undercover_agenten + 0.006*feit + '
  '0.0

# Compute Coherence Score

In [ ]:
# Compute Coherence Score
coherence_model_hdp = CoherenceModel(model=hdp_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_hdp = coherence_model_hdp.get_coherence()
print('\nCoherence Score: ', coherence_hdp)

## LDA to LSA

In [ ]:
def ret_top_model():
    top_topics = [(0, 0)]
    while top_topics[0][1] < 0.97:
        lm = lda_model(corpus=corpus, id2word=id2word)
        coherence_values = {}
        for n, topic in lm.show_topics(num_topics=-1, formatted=False):
            topic = [word for word, _ in topic]
            cm = CoherenceModel(topics=[topic], texts=train_texts, dictionary=dictionary, window_size=10)
            coherence_values[n] = cm.get_coherence()
        top_topics = sorted(coherence_values.items(), key=operator.itemgetter(1), reverse=True)
    return lm, top_topics